In [1]:
import torch
import matplotlib.pyplot as plt
import random

## Callbacks

### Callbacks as GUI events
The most common place to see callbacks in software is for GUI  events. 
The main GUI in Jupyter Notebooks is ipywidgets.

In [2]:
import ipywidgets as widgets

In [3]:
def f(o): print('hi')

From the [ipywidget docs](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Events.html):

- *the button widget is used to handle mouse clicks. The on_click method of the Button can be used to register function to be called when the button is clicked*

We can create a widget like a button, and when we display it, it shows a button.
And at the moment it doesn't  do anything if we click on it.

In [4]:
w = widgets.Button(description='Click me')

In [5]:
w

Button(description='Click me', style=ButtonStyle())

We can  add an `on_click()` callback to it, pass it a function, which is called when you click it. 
`w.on_click(f)` is going to assign the f function to the `on_click` callback.
Now if we click this it's doing it.

In [6]:
w.on_click(f)

*NB: When callbacks are used in this way they are often called "events".*

A callback is a callable (a more general version of a function) that we've provided. 
Above is a function that we provided that will be called back to when something happens, e.g., clicking a button.
So this is how we are defining and using a callback as a GUI event.

### Creating your own callback

Training a model is often a slow calculation.
We would like to print out the loss from time to time or show a progress bar.
For those kinds of things, we define a callback that is called at the end of each epoch or batch or every few seconds. 
<br>
Let's say we've got a slow calculation, as below, because we sleep for a second after each addition.

In [7]:
from time import sleep

In [8]:
def slow_calculation():
    res = 0
    for i in range(5):
        res += i*i
        sleep(1)
    return res

We run our slow calculations, takes a long time to return the answer. 

In [9]:
slow_calculation()

30

Lets modify `slow_calculation` such that we can optionally pass t a callback `cb`.
The code is the same, except we've added one line of code: 
if there's a callback `cb`, then call it and pass in where we're up to `i`.

In [10]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        res += i*i
        sleep(1)
        if cb: cb(i)
    return res

Lets define `show_progress()` a  callback function. 

In [11]:
def show_progress(epoch):
    print(f"Awesome! We've finished epoch {epoch}!")

If we call `slow_calculation` passing in our callback, it's going to call this function at the end of each step.  
So here we've created our own callback. 

In [12]:
slow_calculation(show_progress)

Awesome! We've finished epoch 0!
Awesome! We've finished epoch 1!
Awesome! We've finished epoch 2!
Awesome! We've finished epoch 3!
Awesome! We've finished epoch 4!


30

A callback is not "special", it doesn't require its own syntax.
It's just passing a function which some other function will call at particular times.
We don't have to define the callback ahead of time.
Using `lambda` we can define the function at the same time that we call the slow calculation.

### Lambdas and partials

Lambda defines a function, but it doesn't give it a name. 
Below `slow_calculation` takes one parameter `o` and prints out as before, using a `lambda`.  

In [13]:
slow_calculation(lambda o: print(f"Awesome! We've finished epoch {o}!"))

Awesome! We've finished epoch 0!
Awesome! We've finished epoch 1!
Awesome! We've finished epoch 2!
Awesome! We've finished epoch 3!
Awesome! We've finished epoch 4!


30

We can make it more sophisticated and pass in the `exclamation`.

In [14]:
def show_progress(exclamation, epoch):
    print(f"{exclamation}! We've finished epoch {epoch}!")

And we can have our `lambda` call that function.  

In [15]:
slow_calculation(lambda o: show_progress("OK I guess", o))

OK I guess! We've finished epoch 0!
OK I guess! We've finished epoch 1!
OK I guess! We've finished epoch 2!
OK I guess! We've finished epoch 3!
OK I guess! We've finished epoch 4!


30

`make_show_progress` a function that returns a function, where we pass in the `exclamation`.  
And there's no need to give it a name, just return a function that  calls that exclamation. 

In [16]:
def make_show_progress(exclamation):
    _inner = lambda epoch: print(f"{exclamation}! We've finished epoch {epoch}!")
    return _inner

In [17]:
slow_calculation(make_show_progress("Nice!"))

Nice!! We've finished epoch 0!
Nice!! We've finished epoch 1!
Nice!! We've finished epoch 2!
Nice!! We've finished epoch 3!
Nice!! We've finished epoch 4!


30

Instead of using a Lambda,  we can define an `_inner` function like this.
Below function returns a function and does the same thing.

In [18]:
def make_show_progress(exclamation):
    # Leading "_" is generally understood to be "private"
    def _inner(epoch): print(f"{exclamation}! We've finished epoch {epoch}!")
    return _inner

In [19]:
slow_calculation(make_show_progress("Nice!"))

Nice!! We've finished epoch 0!
Nice!! We've finished epoch 1!
Nice!! We've finished epoch 2!
Nice!! We've finished epoch 3!
Nice!! We've finished epoch 4!


30

In [20]:
f2 = make_show_progress("Terrific")

In [21]:
slow_calculation(f2)

Terrific! We've finished epoch 0!
Terrific! We've finished epoch 1!
Terrific! We've finished epoch 2!
Terrific! We've finished epoch 3!
Terrific! We've finished epoch 4!


30

In [22]:
slow_calculation(make_show_progress("Amazing"))

Amazing! We've finished epoch 0!
Amazing! We've finished epoch 1!
Amazing! We've finished epoch 2!
Amazing! We've finished epoch 3!
Amazing! We've finished epoch 4!


30

We can also do the same thing using `partial` to call `show_progress()` and  pass a string "OK I guess". 
Another example of a function returning a function.

In [23]:
from functools import partial

In [24]:
slow_calculation(partial(show_progress, "OK I guess"))

OK I guess! We've finished epoch 0!
OK I guess! We've finished epoch 1!
OK I guess! We've finished epoch 2!
OK I guess! We've finished epoch 3!
OK I guess! We've finished epoch 4!


30

In [25]:
f2 = partial(show_progress, "OK I guess")

### Callbacks as callable classes

Python doesn't care about types. No requirement that `cb` be a function, just has to be a callable, something that we can call. 
Another way of creating a callable is defining `__call__`.
`ProgressShowingCallback` is a callable object that does the same thing as `make_show_progress()`.
`__init__` stores the exclamation and `__call__` prints.

In [26]:
class ProgressShowingCallback():
    def __init__(self, exclamation="Awesome"): self.exclamation = exclamation
    def __call__(self, epoch): print(f"{self.exclamation}! We've finished epoch {epoch}!")

In [27]:
cb = ProgressShowingCallback("Just super")

In [28]:
slow_calculation(cb)

Just super! We've finished epoch 0!
Just super! We've finished epoch 1!
Just super! We've finished epoch 2!
Just super! We've finished epoch 3!
Just super! We've finished epoch 4!


30

### Multiple callback funcs; `*args` and `**kwargs`

We define a function that has `*args` and `**kwargs`, nothing else, and have it print them.

In [29]:
def f(*args, **kwargs): print(f"args: {args}; kwargs: {kwargs}")

We call the function passing 3, `a`, `b`, and `thing1=”hello”`. 
`3, a, b` are by position, there is no "=".
Arguments that are passed by position are placed in `*args`, (if there is one).

In [30]:
f(3, 'a', 'b', thing1="hello")

args: (3, 'a', 'b'); kwargs: {'thing1': 'hello'}


It doesn't have to be called `args`, can be called anything in the `"*"` star bit.
`args` is a tuple  containing the positionally passed arguments, `kwargs` is a dictionary containing the named arguments. 

In [31]:
def g(a,b,c=0): print(a,b,c)

Rather than just using them as  parameters, we can also use them when calling something. 
Lets have `args= [1, 2]`, and `kwargs` be a dictionary containing `{‘c’: 3}`. 
I can then call `g()`  and I can pass in `*args, **kwargs`.
It is going to take `1, 2`,  and pass them as individual arguments, positionally.
And it's going to take the `{‘c’:  3}` and pass it as a named argument, `c=3`.   

In [32]:
args = [1,2]
kwargs={'c':33}
g(*args, **kwargs)

1 2 33


### Another way to do callbacks:
Passing a class, `PrintStepCallback()` that's not callable, but has 
callable methods: `before_calc` and `after_calc`. 

In [33]:
class PrintStepCallback():
    def __init__(self): pass
    def before_calc(self, *args, **kwargs): print(f"About to start")
    def after_calc (self, *args, **kwargs): print(f"Done step")

In [34]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        if cb: cb.before_calc(i)
        res += i*i
        sleep(1)
        if cb: cb.after_calc(i, val=res)
    return res

We run that and is printing before (after) every step by calling `before_calc()` (`after_calc()`).
 

In [35]:
slow_calculation(PrintStepCallback())

About to start
Done step
About to start
Done step
About to start
Done step
About to start
Done step
About to start
Done step


30

A good use of `*args` and `**kwags` is to "eat up" arguments we do not want.
Below `PrintStatusCallback()` passes to `after_calc()`, the `epoch` number and the value it's up to.
By using `*args` and `**kwags`, we can safely ignore them if we don't want them.
If we didn't have those it would complain, since we do will "chew them" and not complain. 

In [36]:
class PrintStatusCallback():
    def __init__(self): pass
    def before_calc(self, epoch, **kwargs): print(f"About to start: {epoch}")
    def after_calc (self, epoch, val, **kwargs): print(f"After {epoch}: {val}")

In [37]:
slow_calculation(PrintStatusCallback())

About to start: 0
After 0: 0
About to start: 1
After 1: 1
About to start: 2
After 2: 5
About to start: 3
After 3: 14
About to start: 4
After 4: 30


30

### Modifying behavior

Or we could use the arguments, `epoch` and `val` and print them out.
This callback is giving us status as we go. 
Skip this bit because we don't really care about that. 

In [38]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        if cb and hasattr(cb,'before_calc'): cb.before_calc(i)
        res += i*i
        sleep(1)
        if cb and hasattr(cb,'after_calc'):
            if cb.after_calc(i, res):
                print("stopping early")
                break
    return res

In [39]:
class PrintAfterCallback():
    def after_calc (self, epoch, val):
        print(f"After {epoch}: {val}")
        if val>10: return True

In [40]:
slow_calculation(PrintAfterCallback())

After 0: 0
After 1: 1
After 2: 5
After 3: 14
stopping early


14

In [41]:
class SlowCalculator():
    def __init__(self, cb=None): self.cb,self.res = cb,0
    
    def callback(self, cb_name, *args):
        if not self.cb: return
        cb = getattr(self.cb,cb_name, None)
        if cb: return cb(self, *args)

    def calc(self):
        for i in range(5):
            self.callback('before_calc', i)
            self.res += i*i
            sleep(1)
            if self.callback('after_calc', i):
                print("stopping early")
                break

In [42]:
class ModifyingCallback():
    def after_calc (self, calc, epoch):
        print(f"After {epoch}: {calc.res}")
        if calc.res>10: return True
        if calc.res<3: calc.res = calc.res*2

In [43]:
calculator = SlowCalculator(ModifyingCallback())

In [44]:
calculator.calc()
calculator.res

After 0: 0
After 1: 1
After 2: 6
After 3: 15
stopping early


15

## `__dunder__` thingies

Anything that looks like `__this__` is, in some way, *special*. Python, or a library, can define some functions that they will call at certain documented times. 
For instance, when a class sets up a new object, Python will call `__init__`. 
Python's [Data Model - Special method names](https://docs.python.org/3/reference/datamodel.html#special-method-names).
For instance, if Python sees `+` it will call the special method `__add__`. 

Lets define a class that's bad at adding, it always adds 0.01 to it.
SloppyAdder(1) + SloppyAdder(2)  equals 3.01. 
“+” here is actually calling `__add__`. 

In [45]:
class SloppyAdder():
    def __init__(self,o): self.o=o
    def __add__(self,b): return SloppyAdder(self.o + b.o + 0.01)
    def __repr__(self): return str(self.o)

In [46]:
a = SloppyAdder(1)
b = SloppyAdder(5)
a+b

6.01

Special methods to know well (see [data model](https://docs.python.org/3/reference/datamodel.html#object.__init__)) are:

- [`__getitem__`](https://docs.python.org/3/reference/datamodel.html#object.__getitem__)
- [`__getattr__`](https://docs.python.org/3/reference/datamodel.html#object.__getattr__)
- [`__setattr__`](https://docs.python.org/3/reference/datamodel.html#object.__setattr__)
- [`__del__`](https://docs.python.org/3/reference/datamodel.html#object.__del__)
- [`__init__`](https://docs.python.org/3/reference/datamodel.html#object.__init__)
- [`__new__`](https://docs.python.org/3/reference/datamodel.html#object.__new__)
- [`__enter__`](https://docs.python.org/3/reference/datamodel.html#object.__enter__)
- [`__exit__`](https://docs.python.org/3/reference/datamodel.html#object.__exit__)
- [`__len__`](https://docs.python.org/3/reference/datamodel.html#object.__len__)
- [`__repr__`](https://docs.python.org/3/reference/datamodel.html#object.__repr__)
- [`__str__`](https://docs.python.org/3/reference/datamodel.html#object.__str__)

### `__getattr__` and `getattr`

`getattr` is the opposite of `setattr`.
`A` is a class with two attributes, `a` and `b`, set to 1 and 2. 
We create an object of that class, `a.b` equals 2, because we set b to 2. 
`a.b` is Python syntax sugar, calling (behind the scenes) `getattr` on the object. 
`a.b` is the same as `getattr(a, ‘b’)`.

In [47]:
class A:
    a,b=1,2

In [48]:
a = A()

In [49]:
a.b

2

In [50]:
getattr(a, 'b')

2

Python is a dynamic language, we can even set it up so we don't know what attributes are going to be called. This can be fun/crazy,  because we can call getattr a, and then either ‘b’ or ‘a’ randomly.

In [51]:
getattr(a, 'b' if random.random()>0.5 else 'a')

2

`getattr` behind the scenes calls `__getattr__`, by default, the version in the object base class. 
Class `B` is like `A`, got `a` and `b` defined, and also got `__getattr__`  defined. 
`__getattr__` is only called for stuff that hasn't been defined yet, 
and it'll pass in the key or the name of the attribute.  
Generally speaking, if the first character is an underscore, it's going to be private or special,
so we raise an  attribute error. 
Otherwise we "steal it" and return f‘Hello from {k}’. 

In [52]:
class B:
    a,b=1,2
    def __getattr__(self, k):
        if k[0]=='_': raise AttributeError(k)
        return f'Hello from {k}'

In [53]:
b = B()

`b.a` is defined, returns 1. 

In [54]:
b.a

1

`b.foo` is not defined, so it calls `__getattra__` and we get back "hello from foo". 
This gets used a lot to make it more convenient to access things. 

In [55]:
b.foo

'Hello from foo'

In [56]:
b._foo

AttributeError: _foo

## Browsing source code and debugging

- Jump to tag/symbol by with (with completions)
- Jump to current tag
- Jump to library tags
- Go back
- Search
- Outlining / folding